<div style="background-color; color: black; width:95%; vertical-align: middle; padding:15px; margin: 10px; border-radius: 10px">

# Topography based models

<div style="background-color; color: black; width:95%; vertical-align: middle; padding:15px; margin: 10px; border-radius: 10px">

## The Ourthe catchment

We will focus specifically on the catchment of the Ourthe upstream of Tabreux (1600 km<sup>2</sup>). Both topography and geomorphology play an important role in the hydrological response of the Upper Ourthe catchment. The catchment is situated in the hilly plateaus of the Ardennes in the eastern part of Belgium "a well-known area for those who followed Hydrogeology" (Figure 1). The elevation of the catchment varies between 150 and 650 m above sea level. The soil mainly consists of sandstone, shale and limestone, which have been eroded by small river systems that drain into the river Meuse. The region can be characterized as mostly rain-fed with some snow during the winter period. This results in a highly variable runoff regime with low discharges in summer and high peaks in winter.


<div style="background-color; color: black; width:95%; vertical-align: middle; padding:15px; margin: 10px; border-radius: 10px">

## Digital elevation models for hydrological applications

The data and scripts can be found [here](../zip_files/CCH_pcprac_4). Download and unzip. This will generate the folder ``topmodels``, containing the subdirectories ``R_scripts``, `DEM` and `Data`. 


### DEM

The DEM you will use today was obtained by the Shuttle Radar Topography Mission (SRTM) which generated a near global elevation map. For Europe the resolution of this dataset is about 90 meter and can be downloaded from the internet (http://www2.jpl.nasa.gov/srtm/). Your DEM file is called ``ourthe_90m.asc`` and is located in the folder `DEM`. 

```{figure} ../images/topomap.png
---
width: 70%
name: topomap
align: center
---
(Location of the Ardennes and a 200x200-km box with a topographic map, the main channel network (black), the Ourthe catchment boundary (white), the catchment outlet ($\blacktriangle$), the meteorological station ($\circ$), rain gauges (+) and weather radar ($\bullet$).)
```
 
---
Open this file in Notepad and find out how this ASCII file is set up. What do the numbers in the big matrix mean?

<details>
  <summary>Show Answer</summary>
  <span style="color:blue">
 The elevation of each pixel.
  </span>
</details>

---

### Preprocessing

Before you can use the DEM for hydrological applications, the DEM has to be preprocessed. In this preprocessing step, pits in the topography field are filled, flow directions are computed and contributing areas and slope gradients are computed.

For this preprocessing, multiple GIS programs can be used (e.g. ArcGIS, SAGA GIS, GRASS). For this practical these preprocessing steps have already been performed using the TAUDEM package. TAUDEM is a terrain analysis package for ArcGIS developed by D.G. Tarboton of Utah State University. For more information concerning the specific aspects of TAUDEM, see _http://hydrology.usu.edu/taudem/taudem5.0/_ `index.html`. The following sections explain what happens during the preprocessing phase.

#### Sink / pit filling

The main assumptions behind spatial hydrological analyses is that water will flow from higher to lower cells. To obtain a continuous flow direction map, the lowest cell in a DEM should be situated at the boundary of a DEM. Unfortunately, local topography or erroneous measurements by the satellite can result in local depressions (also called sinks or pits). Because the cells surrounding the pits are higher in elevation, water will never flow out of them and will never reach the river. 

Although sinks may represent real terrain features, they hinder the analysis of drainage networks. Local depressions should therefore be filled before making a flow direction map. Using TAUDEM, the original DEM file ``ourthe_90m.asc`` was corrected for pits, resulting in a new DEM file ``ourthe_90mfel.asc``, which can also be found in the `DEM` folder.

#### Flow direction, slope and contributing area

There are many ways to compute flow directions. In this practical the simplest and oldest method will be applied: the D8 flow direction method. The elevation of each pixel is compared to the elevations of the 8~neighbouring cells and water will flow in the direction of the steepest descent:

```{figure} ../images/DEM.png
---
name: topomap
align: center
width: 70%
---
```

The flow directions are represented by numbers:

```{figure} ../images/D8_flow_directions.png
---
width: 70%
name: D8_flow_directions
align: center
---
```

Boundary pixels for which no downstream direction could be obtained receive the value of $-1$. The output of this step was saved in the file `ourthe_90mp.asc`. Once these directions have been determined, TAUDEM immediately generates the file ``ourthe_90msd8.asc``, containing the local slopes for all pixels. From the flow direction map, the upstream area for each pixel can be computed. TAUDEM starts at the highest points of the DEM and moves downward. In ``ourthe_90mad8.asc`` the number of upstream pixels is given for each pixel.

--- 

``ourthe_90mp.asc``, ``ourthe_90msd8.asc`` and ``ourthe_90mad8.asc`` in Notepad and try to understand what the numbers mean.

---

### R for DEM analyses

After these preprocessing steps, R can be used for spatial hydrological analyses. Open RStudio and load the script ``topmodels.R``. Set the working directory and run the part under ``Getting started``.

The folder ``R_scripts`` contains more scripts with functions used in the main R script ``topmodels.R`` (you don't have to open these scripts now):

* `SetGeo.R` gives pixel size and extent of the DEM.

* `nbrtable.R` Within R a DEM matrix is internally represented as a vector. Therefore, for every pixel within the grid this function calculates the vector index value for the 8 neighbour points.

* ``downnbr.R`` gives the vector index number of a pixel's downstream neighbour based on the D8 flow direction method.

* ``mainbasin.R`` gives the upstream area for a given pixel.

* ``GetNewCoordinates.R`` zooms in to the catchment when the catchment area covers a small part of the DEM.

* ``flowdistance.R`` calculates the distance to the channel network for each pixel.

* ``plotflowdistance.R`` draws a map of the distance to the channel for each pixel obtained by ``flowdistance.R``.

* ``channeldistance.R`` gives the distance to the outlet (measured along the channel) for each pixel (given a certain channel network).

* ``channelpixelsize.R`` computes for each channel pixel how many metres of channel it contains.

* ``strahler_order.R`` gives Strahler order numbers for each channel link.

* ``shreve_order.R`` gives Shreve order numbers for each channel link.

* ``TOPMODEL.R`` runs the hydrological model TOPMODEL.

* ``NSeff.R`` computes the Nash-Sutcliffe efficiency.

Run the part under ''Extra functions''. You then load all extra functions. For example, when you run the line source(``"R_scripts/SetGeo.R"``) in the ``topmodels.R``-script, R runs the whole ``SetGeo.R``-script at once in the background. Because the ``SetGeo.R``-script contains the definition of the function ``SetGeo``, this function with the argument ``A`` appears in the workspace window.


<div style="background-color; color: black; width:95%; vertical-align: middle; padding:15px; margin: 10px; border-radius: 10px">

## DEM analyses

###  Read data 

Next, run the part of the script under "Read Data". This part of the script generates a map with the pit filled DEM. Check if your figure is similar to Figure {numref}`topomape`.

Read the part of the script under "Catchment Area". If you run this part in R, only the part of the DEM that lies upstream of Tabreux is selected. This part belongs to the Upper Ourthe catchment. 

---
Click on `basin` in the workspace window. What do the TRUE and FALSE mean?

<details>
  <summary>Show Answer</summary>
  <span style="color:blue">
  
 TRUE: Inside the catchment.
 FALSE: Outside the catchment.
  </span>
</details>

---

Make sure you understand what happens in the line `Elevation[basin == FALSE] = NA`. If you don't know what the square brackets do, reread Section 11.1 of the (very) short introduction to R (which can be found on the R website): 
`cran.r-project.org/doc/contrib/Torfs+Brauer-Short-R-Intro.pdf`.

---
How many pixels belong to the Ourthe catchment and what is the corresponding area?

<details>
  <summary>Show Answer</summary>
  <span style="color:blue">
  
 The total number of pixels corresponding to the catchment is 199886, which corresponds to 1619 km<sup>2</sup>
  </span>
</details>

---



### Channel network


Because you do not have information on where the channels are (which is almost always the case), you will predict the location of the channels with the DEM. Assume that a river branch will develop when the upstream area exceeds a certain value representing the soil's flow capacity; excess water will start to flow overland instead, creating a channel. Assume that for a pixel to be part of the channel network, the upstream area has to be equal to or exceed 0.1~$\text{km}^2$ ($A_c$). 

Run the part of the script under "Channel Network". This will generate the channel network. Click on "Channel" in the workspace window to see what this matrix looks like.

---
How many pixels are part of the channel network?

<details>
  <summary>Show Answer</summary>
  <span style="color:blue">
  
 29177

  </span>
</details>


\How many pixels are part of the network when you change the upstream area to 0.03, 0.3 and 1 $\mathrm{km}^2$.

<details>
  <summary>Show Answer</summary>
  <span style="color:blue">

  0.03 $\mathrm{km}^2$:         70973 

  0.3 $\mathrm{km}^2$:          17288

  1 $\mathrm{km}^2$:            10477

</span>
</details>

What is the relation between channel density and $A_c$?

```{figure} ../images/Question3.png
---
width: 70%
name: Question3 
align: center
---
```
<details>
  <summary>Show Answer</summary>
  <span style="color:blue">

  The channel density decreases when you increase Ac (the upstream area which should be exceeded in order to form a channel).

</span>
</details>

---

Set the upstream area to 0.1 $\mathrm{km}^2$ for the next assignments.

---


### Channel network ordering


The shape of channel network can be analyzed with several ordering systems. Today we will focus on the Strahler (left) and Shreve (right) ordering systems:


```{figure} ../images/Shreve.png
---
width: 70%
name: Shreve 
align: center
---
```

Use the function `shreve` to calculate the Shreve order for each channel pixel. 

---

What is the maximum Shreve value? (Look at the arguments of the function `max` to find out how to exclude non-channel pixels in the computation.)


```{figure} ../images/Question11.png
---
width: 70%
name: Question11 
align: center
---
```

<details>
  <summary>Show Answer</summary>
  <span style="color:blue">

  4183

</span>
</details>

 What does this maximum represent?

<details>
  <summary>Show Answer</summary>
  <span style="color:blue">

 This is the number of first order channels (and sources).  

</span>
</details>

---

Do the same for the Strahler order.

---

What is the maximum Strahler number?

<details>
  <summary>Show Answer</summary>
  <span style="color:blue">
 7  

</span>
</details>

How does this value change when you increase $A_c$; the upstream area which defines the channel network? (what you did in [Section 6.3.3.2](#channel-network))

```{figure} ../images/Question12.png
---
width: 70%
name: Question12 
align: center
---
```

<details>
  <summary>Show Answer</summary>
  <span style="color:blue">

  When the upstream area increases, the Strahler number will decrease,
  because there will be fewer branches. 

</span>
</details>


### Drainage density and hillslope length

The function `channelpixelsize` calculates for every channel pixel how many metres of channel it contains.

--- 
Click on `Channel_Length_Per_Pixel` in the workspace window. What do NA, 90 and 127 tell you about the flow direction?

<details>
  <summary>Show Answer</summary>
  <span style="color:blue">

 NA:  There is no river on this pixel

 90:  The river flows horizontally or vertically through this pixel.
 
 127: The river flows diagonally through this pixel.

</span>
</details>

Where do the numbers 90 and 127 come from?

<details>
  <summary>Show Answer</summary>
  <span style="color:blue">

 Size of the pixel is 90 m and $127 \sim \sqrt(90^2 + 90^2)$

</span>
</details>

---


A rectangular valley with a channel in the middle and the local water divide at the edges, can be characterized by several measures: the channel length $L_c$, the area $A$, the hillslope length $L_h$ and the drainage density $d$:

```{figure} ../images/hillslope.png
---
width: 70%
name: hillslope 
align: center
---
```

Give the equation to compute the area of the valley in the figure above from $L_h$ and $L_c$.?

<details>
  <summary>Show Answer</summary>
  <span style="color:blue">

 $A$ = $L_c\cdot2L_h$

</span>
</details>



Give the equation to compute the drainage density from area $A$ and channel length $L$.

<details>
  <summary>Show Answer</summary>
  <span style="color:blue">

 $d$=$L_c/A$

</span>
</details>

Give the equation to compute the drainage density from the hillslope length $L_h$.

<details>
  <summary>Show Answer</summary>
  <span style="color:blue">

 $d$=$1/(2L_h)$

</span>
</details>

Give the equation to compute $L_h$ from $d$.

<details>
  <summary>Show Answer</summary>
  <span style="color:blue">

 $L_h$=$1/(2d)$

</span>
</details>

What is the unit of drainage density?

<details>
  <summary>Show Answer</summary>
  <span style="color:blue">

 Drainage density is defined as the total channel length over the total area ($L_c/A$). The unit is m divided by $\mathrm{m}^2$, so $\mathrm{m}^{-1}$.

</span>
</details>

Compute the drainage density of the Ourthe catchment (for $A_c=0.1$)

<details>
  <summary>Show Answer</summary>
  <span style="color:blue">

 $d$=$0.002$~ $\mathrm{m}^{-1}$

</span>
</details>

What is the average hillslope length?

<details>
  <summary>Show Answer</summary>
  <span style="color:blue">

 262 m. Of course the Ourthe catchment is not a collection of rectangular sub-catchments, but this method does give an idea of the average hillslope length.

</span>
</details>

---

### Distance to outlet

Compute for each pixel the distance to the outlet. This is preprogrammed in `channeldistance`. 

---

What is the distance from the most upstream channel pixel to the outlet?

```{figure} ../images/Question7.png
---
width: 70%
name: Question7 
align: center
---
```
<details>
  <summary>Show Answer</summary>
  <span style="color:blue">

 115 km

</span>
</details>

 Plot the average channel elevation versus distance to the outlet. What do the slopes of the curves show?


<details>
  <summary>Show Answer</summary>
  <span style="color:blue">

 The slope of the channel bed.

</span>
</details>

What can you say about the relation between the slopes of the curves and the distance to the outlet?

<details>
  <summary>Show Answer</summary>
  <span style="color:blue">

 The slope of the channel bed.

</span>
</details>

```{figure} ../images/Question8.png
---
width: 70%
name: Question8 
align: center
---
```

<details>
  <summary>Show Answer</summary>
  <span style="color:blue">

 Most of the lower order channel links (upstream channel links) have a steeper slope than the higher order ones. The result of this is that close to the sources, the water flows more quickly (when all other factors such as bed roughness and vertical cross-section of the river are considered equal, which is of course not the case as the downstream stretches of river are wider and deeper). 

</span>
</details>

---

### Channel width function

From the channel network you can compute the network width function. This function gives the fraction of channel pixels at a given distance from the outlet. Plot the width function.

---

Explain the shape of the width function with the shapes of the channel network and catchment.

```{figure} ../images/Question13.png
---
width: 70%
name: Question13 
align: center
---
```

<details>
  <summary>Show Answer</summary>
  <span style="color:blue">

 Close to the outlet (20-40~km) there are many channels, at about 70~km from the outlet there are few channels and between 80 and 100~km from the outlet there are again more channels. From the catchment maps, you can see that the Ourthe catchment has a narrow part in the middle, which causes the low number of links (channel pixels) in the width function.

</span>
</details>

---

### Channel travel time distribution

The velocity of the water in the river is generally between 0.3 and 3.0 m $\mathrm{s}^{-1}$. Here a value of 0.8 m $\mathrm{s}^{-1}$ is assumed for every stretch of river. With the flow velocity you can convert the distance-to-outlet-map into a channel travel time map.

---
What can you say about the pattern and variability in flow times and their relation to the shape of the catchment and the channel network?

<details>
  <summary>Show Answer</summary>
  <span style="color:blue">

 The spatial pattern in the channel travel time map is the same as in the distance-to-outlet-map. To make the travel-time-map, you multiplied the distance-to-outlet at every pixel with a constant, so nothing happened to the spatial distribution.

</span>
</details>

---
Different channel velocities lead to different travel time distributions. Compute the maximum and median travel time with different channel velocities ranging from $0.3$ to $3.0$ m $\mathrm{s}^{-1}$ and plot the results (channel velocity on the $x$-axis and travel time on the $y$-axis).

---

What is the relation between flow velocity and maximum channel travel time?

```{figure} ../images/Question9.png
---
width: 70%
name: Question9 
align: center
---
```

<details>
  <summary>Show Answer</summary>
  <span style="color:blue">

 When the channel velocity doubles, the maximum travel time is halved.

</span>
</details>


If all rain would fall on the channel network directly, what do the maximum and median travel time then represent?

<details>
  <summary>Show Answer</summary>
  <span style="color:blue">

 The time necessary to discharge all or 50\% of the water out of the catchment.

</span>
</details>

---

<div style="background-color; color: black; width:95%; vertical-align: middle; padding:15px; margin: 10px; border-radius: 10px">

## Rainfall-runoff modelling

### Time series

Before you start modelling, you should get a better feeling of the water balance terms in the Ourthe catchment. The file ``PEQ_Ourthe_2002_2003.txt`` in the folder `Data` contains 2 years of basin averaged hourly precipitation [$mm$ $\mathrm{h}^{-1}$ ], potential evaporation [$mm$ $\mathrm{h}^{-1}$] and discharge [$\mathrm{m}^{3}$ $\mathrm{s}^{-1}$] data. Use the year 2002 for calibration and Feb.-Dec. 2003 for validation. The discharge data of January 2003 are removed - you will predict these in the final assignment.

Comparing fluxes is easier when they have the same units. Use the catchment area to convert the discharge to mm $\mathrm{h}^{-1}$ and plot the time series. To remove the hourly variability, the data can be averaged over monthly periods. This is preprogrammed in the script. In addition, the $\mathrm{10}^{th}$ and $\mathrm{90}^{th}$ percentiles are computed.

---

Look at the seasonal variation to get a rough idea of the dominant fluxes during different seasons.

```{figure} ../images/Question4.png
---
width: 70%
name: Question4 
align: center
---
```

```{figure} ../images/Question5.png
---
width: 70%
name: Question5 
align: center
---
```

The models in [Chapter 6.3.4](#rainfall-runoff-modelling) need input, but cannot deal with negative input ($ET>P$), because they do not contain stores from which water can be removed (they use transfer functions instead of reservoirs). You could just use $P$ as input, but of course part of the precipitation evaporates. As a quick fix, a factor $f$ is introduced to close the water balance over 2002 to account for evapotranspiration: 



$$
  f \cdot \Sigma P - \Sigma Q = 0
$$ (equation_1)

Hourly values of effective precipitation are computed as

$$
  P_\mathrm{eff} = f \cdot P
$$ (equation_2)


### Channel travel time distribution model

From the channel width function in [section 6.3.3.6](#channel-width-function) you can make a simple rainfall-runoff model. In this model you assume that all precipitation enters the channel network directly.

The function ``Q_geomorph_channel`` converts the width function to a travel time distribution model with a convolution integral. Try to understand the different aspects of this function (look in the file ``Q_geomorph_channel.R``).

---

How does the simulated discharge compare to the observed discharge? What happens if you change the velocity?

```{figure} ../images/Question14.png
---
width: 70%
name: Question14 
align: center
---
```

<details>
  <summary>Show Answer</summary>
  <span style="color:blue">

 With a velocity of 0.8 m $\mathrm{s}^{-1}$ the response is too flashy, while for very slow values (0.008 m $\mathrm{s}^{-1}$) there is no clear response of runoff to rainfall

</span>
</details>

---

### Hillslope width function

Of course, the assumption that all precipitation immediately enters the channel network is not valid: the simulated discharge response is too fast. This illustrates that the response of a catchment is not only characterized by the channel network but also by the soil, slope and vegetation on the hillslopes.

--- 
Compute for each pixel the distance to the nearest channel, plot the map and make a histogram: the hillslope width function.


<details>
  <summary>Show Answer</summary>
  <span style="color:blue">

 ```{figure} ../images/Question15.png
 ---
 width: 70%
 name: Question15 
 align: center
 ---
 ```

</span>
</details>

What is the average distance to the channel?

<details>
  <summary>Show Answer</summary>
  <span style="color:blue">

 250 m 

</span>
</details>

 How does the average distance to the channel compare to the average hillslope length you computed in [Section 6.3.3.4.](#drainage-density-and-hillslope-length)?

<details>
  <summary>Show Answer</summary>
  <span style="color:blue">

 They compare quite well.

</span>
</details>

What information does this width function give?

```{figure} ../images/Question15_2.png
---
width: 70%
name: Question15_2 
align: center
---
```

<details>
  <summary>Show Answer</summary>
  <span style="color:blue">

 Most pixels are close to a channel and the number of pixels at a certain distance from a channel decreases with that distance to the channel. There are few pixels more than 100~m away from the nearest channel.

</span>
</details>

---

### Channel and hillslope travel time distribution model

A catchment's travel time distribution is determined by the time a water droplet spends in both the hillslope and the channel network. You already computed the channel travel time distribution from the channel width function ([Section 6.3.4.2](#channel-travel-time-distribution-model)). In the same way, you can make a hillslope travel time distribution from the histogram of the distances to the nearest channel. Just as in the channel, we assume that the flow velocity in the hillslope is constant.

---

Why is the assumption of a constant (no variation in time) flow velocity in the hillslope not realistic?

<details>
  <summary>Show Answer</summary>
  <span style="color:blue">

 Water travels overland, through macropores and the unsaturated and saturated zone matrix. Between these flow routes, velocity can easily differ by a factor 100. The distribution of water over these flowpaths varies temporally.

</span>
</details>

---

The functions ``Q_geomorph_hillslope`` and ``Q_hillslope_parts`` have a similar set-up as the previously used function ``Q_geomorph_channel``. As a first estimate, the effective $\mathrm{}^{*}$ velocity in the hillslope is assumed to be 0.1m $\mathrm{s}^{-1}$ and the channel network velocity 0.8~m $\mathrm{s}^{-1}$.

$\mathrm{}^{*}$ The word effective is added because the actual flow velocity of the water particles through the soil is much slower than the response caused by pressure changes.

---

During which part of the year do the simulated discharge peaks fit the observed ones well? Why?


```{figure} ../images/Question16.png
---
width: 70%
name: Question16 
align: center
---
```

<details>
  <summary>Show Answer</summary>
  <span style="color:blue">

 The simulated model performs best during the winter period, but, because the amount of evaporation was assumed to a fixed percentage of the amount of precipitation, even during the winter year the response is underestimated.

</span>
</details>

What happens to the baseflow? Why?

<details>
  <summary>Show Answer</summary>
  <span style="color:blue">

 Baseflow results from very slow runoff processes. Because only one value for the flow through the hillslope is taken into account, baseflow is not simulated. In general, it can be expected that baseflow results from much smaller velocities in the hillslope than modelled here.

</span>
</details>

---

Try different values for the effective hillslope velocity.

---

Which values for the effective hillslope velocity yield good results in the winter period and which in the summer?


<details>
  <summary>Show Answer</summary>
  <span style="color:blue">

 Many values possible.

</span>
</details>

You probably found that in winter the hillslope velocity is higher than in in summer. Explain this with the dominant processes that occur during winter.


<details>
  <summary>Show Answer</summary>
  <span style="color:blue">

 As mentioned in the previous question, this model did not take the yearly behaviour in the observed evaporation into account. Therefore, the effective precipitation in the winter (summer) period is too small (large). For very small hillslope velocity values, the summer response is fairly well simulated.

</span>
</details>

---

### Flow routes

The route a water droplet takes through the hillslope to the channel depends on many factors, such as soil type, presence of macropores, vegetation and catchment wetness. Because many of these factors change during the year, the flow routes change, and therefore the travel times change.

Make the model a bit more complex by assuming that the hillslope can be modelled with three different hillslope velocity values, $v=10^{-3}$, $v=10^{-5}$ and $v=10^{-7}~\mathrm{m\,s^{-1}}$, indicating the fast, intermediate and slow hillslope response. Run the model with the defined fractions for fast, medium and slow response velocities.

---

Play around with the fractions of the different velocities. Which fractions did you get for which part of the year?

```{figure} ../images/Question17.png
---
width: 70%
name: Question17 
align: center
---
```

<details>
  <summary>Show Answer</summary>
  <span style="color:blue">

 Many answers possible.

</span>
</details>

For which part of the year were you able to obtain proper discharge estimates?

<details>
  <summary>Show Answer</summary>
  <span style="color:blue">

 Probably in winter.

</span>
</details>

---

<div style="background-color; color: black; width:95%; vertical-align: middle; padding:15px; margin: 10px; border-radius: 10px">

## TOPMODEL

As you probably noticed, the simple model was only able to simulate a proper response behaviour for some events. It is very difficult to simulate the hydrological response of a catchment with a few fixed velocities. It is impossible to grasp the temporal variations in the complex hydrological interactions between vegetation and the unsaturated and saturated zones using this simple approach.

The next step is to use a more complex (but still relatively simple) hydrological model: TOPMODEL. TOPMODEL is a well-known model based on the topographic properties of the catchment.

### Topographic index

The main assumption behind TOPMODEL is that the response behaviour of a given pixel can be estimated by its topographic index value $\lambda$:

$$
  \lambda = \ln \left( \frac{a}{\tan \beta}\right)\
$$(equation3)

where $a$ is the upstream area per unit contour length (pixel length) [m] and $\beta$ the local slope [$^\circ$]. More information on TOPMODEL is given in the reader.

Use the script to compute and plot the topographic index for each pixel.

---

How are the topographic indices distributed?

```{figure} ../images/Question18.png
---
width: 70%
name: Question18
align: center
---
```

<details>
  <summary>Show Answer</summary>
  <span style="color:blue">

 Most cells have a small topographic index value, indication either a small upstream area or large local slope. Both of these characteristics can expected to be observed further away from the channel sections.

</span>
</details>

--- 


### Running TOPMODEL

To run TOPMODEL, you need to specify 5 parameters and 2 initial conditions.
TOPMODEL has already been calibrated for the Ourthe catchment and 20 parameter sets are given in ``TOPMODEL_Parameters.dat``. The initial conditions are the unsaturated zone deficit (`Dbar`) and  root zone deficit (`Drzone`), both effective catchment values.

First, use only the first parameter set (first row in ``TOPMODEL_Parameters.dat``). Run TOPMODEL and simulate the discharge response. Choose the period yourself.

---

Describe the performance of the model. When did the model succeed and when and how did it fail?


<details>
  <summary>Show Answer</summary>
  <span style="color:blue">

 TOPMODEL is quite well able to simulate the hydrologic response of the Ourthe catchment. However looking more closely, the simulated discharge is too large (small) at the beginning (end) of the winter. These latter properties are a consequence of the assumption of a fixed upstream recharge area, which is practice varies over time.

</span>
</details>

---

### Effect of parameters

To analyze the effect of uncertainty in the parameter values, run the model with all 20 parameter sets and make an ensemble of the simulations. This may take a few minutes.

---

When is the parameter uncertainty large? In other words, when is the spread of the grey lines, caused by different parameter sets, large?


```{figure} ../images/Question20.png
---
width: 70%
name: Question20 
align: center
---
```

<details>
  <summary>Show Answer</summary>
  <span style="color:blue">

 The parameter uncertainty is especially large during the first rising limb (Oct-Nov).

</span>
</details>

---

### Effect of initial conditions


The results show that it is especially difficult to simulate the response of the Ourthe for the first main runoff peak in November 2002. This is caused by a wrong estimation of the initial storage, given by `Dbar` and `Drzone`


Change the values of `Dbar` and `Drzone` and run the model again.

---

What is the effect of the initial conditions?

```{figure} ../images/Question21.png
---
width: 70%
name: Question21 
align: center
---
```

<details>
  <summary>Show Answer</summary>
  <span style="color:blue">

 The initial values determine how "wet" the catchment is at the start of the modelling period. When you assume that the catchment is wetter than it really is, you will overestimate the discharge response in the beginning of the period. After a few months, the saturated zone and rootzone deficit will go down to the ``real" values, because when you start too wet, you will get more discharge and you will get a sort of equilibrium value (Dbar and Drzone have gone up and down again and the influence of the initial values becomes less important).

</span>
</details>

---

```{figure} ../images/Question22.png
---
width: 70%
name: Question22 
align: center
---
```
<details>
  <summary>Show Answer</summary>
  <span style="color:blue">

 The best results are obtained from either large (small) values of the saturated zone deficit (Dbar) and small (large) the root zone deficit (Drzone). In case both would have a really large (small) deficit value, the initial storage of the Ourthe would be under (over) estimated.

</span>
</details>

---

Some models use a warming-up period to compensate for bad initial conditions. For example, when you want to forecast discharge from April to September, you may start the simulations in March and neglect the output of the first month.

---

How long would you advise the warming-up period to be for TOPMODEL in the Ourthe catchment and why?

<details>
  <summary>Show Answer</summary>
  <span style="color:blue">

 The effect of the initial conditions is visible for about 3.5 months. After this, the simulations with different initial conditions are similar.

</span>
</details>


What is the relation between `Dbar` and `Drzone`?


<details>
  <summary>Show Answer</summary>
  <span style="color:blue">

 In reality, there usually is much water in the root zone when the saturated zone is full. In the model, when you increase both, you will overestimate discharge and when you decrease both you underestimate. In the middle there is an area where you can exchange Dbar and Drzone: you can increase Dbar a little and decrease Drzone to get the same total storage leading to more or less the same discharge response and therefore the same Nash-Sutcliffe efficiency.

</span>
</details>

---

### Final assignment

Predict the discharge peak in January 2003 using TOPMODEL. Run TOPMODEL with different settings. You can change (1) the warming-up period, (2) parameter sets and (3) initial conditions. Collect the computed discharge from different runs as columns in a matrix to get an ensemble of forecasts, which you can plot and analyse. Follow the instructions in the script.

Compute the 25th, 50th and 75th percentile of the ensemble and cut out the values of January (so 3 time series with 744 values). Save the output as a CSV file in your working directory. Open the file in Excel, copy and paste it in Google Sheets before 13:00. We will discuss the results at 13:05.

